<a href="https://colab.research.google.com/github/priyankaghosh2020/Colab/blob/master/EnergyDisaggregationDemo_View.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# License

Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at . 

      http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing,  
software distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# Pre-work

In [ ]:
# @title Upload files (skip this if this is run locally)

# Use this cell to update the following files
#   1. requirements.txt
#   2. e2e_demo_credential.json
from google.colab import files
uploaded = files.upload()

Saving e2e_demo_credential.json to e2e_demo_credential.json


In [ ]:
# @title Install missing packages

# run this cell to install packages if some are missing
!pip install -r ./requirements.txt

     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 696kB 9.6MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 13.8MB 237kB/s 
     |████████████████████████████████| 26.3MB 117kB/s 
     |████████████████████████████████| 12.6MB 249kB/s 
     |████████████████████████████████| 215kB 55.4MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
     |████████████████████████████████| 2.1MB 47.1MB/s 
     |████████████████████████████████| 92kB 13.9MB/s 
     |████████████████████████████████| 112kB 55.4MB/s 
  Created wheel for gcsfs: filename=gcsfs-0.2.0-cp36-none-any.whl size=26894 sha256=c9221b0dcabd6008fd12426898e36638aacda629a0a1f36347dda90c8129866a
  Stored in directory: /

In [ ]:
!pip install pandas==1.0.5
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-cloud
!pip install --upgrade google-cloud-pubsub

     |████████████████████████████████| 10.1MB 5.2MB/s 
  Found existing installation: pandas 1.0.4
    Uninstalling pandas-1.0.4:
      Successfully uninstalled pandas-1.0.4


     |████████████████████████████████| 174kB 7.5MB/s 
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.15.0 which is incompatible.
ERROR: google-cloud-storage 1.18.1 has requirement google-resumable-media<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0
  Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Found existing ins

     |████████████████████████████████| 143kB 7.8MB/s 
  Created wheel for grpc-google-iam-v1: filename=grpc_google_iam_v1-0.12.3-cp36-none-any.whl size=18500 sha256=fc7311017e25f686c639866c889edf97031b0728eb2cdb09e1cbb9318021ea62
  Stored in directory: /root/.cache/pip/wheels/de/3a/83/77a1e18e1a8757186df834b86ce6800120ac9c79cd8ca4091b
Successfully built grpc-google-iam-v1


In [ ]:

!pip install --upgrade pip
!pip install --upgrade protobuf

     |████████████████████████████████| 1.5MB 9.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 1.3 MB 8.2 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import json
import numpy as np
import os
import pandas as pd
import pandas_gbq
import seaborn as sns
import time
import queue as queue
from google.cloud import pubsub_v1
from IPython import display

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# @title Configurations

# project related
GOOGLE_CLOUD_PROJECT = 'e2e-status-monitoring' #@param
GOOGLE_APPLICATION_CREDENTIALS = 'e2e_demo_credential.json' #@param
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_APPLICATION_CREDENTIALS

# data related
DATASET_ID = 'EnergyDisaggregation'

# pubsub related
PRED_TOPIC = 'pred'
SUB_NAME = 'sub1'
DEVICE_ID = 'e2e-status-monitoring-device-1' #@param

In [ ]:
!pip install --upgrade pyarrow

     |████████████████████████████████| 63.8 MB 58 kB/s 
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


# BQ data loading

In [ ]:
# @title Utility

def get_appliance_info(project_id, dataset):
    """Get appliance info from the big query table.
    
    Load appliance info from big query table.
    
    Args:
      project_id: str, google cloud project id.
      dataset: str, name of the dataset.
    Returns:
      pandas.DataFrame, appliance info.
    """
    res = pandas_gbq.read_gbq(
        'SELECT * FROM {}.ApplianceInfo'.format(dataset),
        project_id,
        dialect='legacy')
    return res

def load_ground_truth(project_id, dataset, appliance_id):
    """Load ground-truth data of a specified appliance from big query table.

    Load true appliance status from big query data.

    Args:
        project_id: str, google project id.
        dataset: str, dataset name.
        appliance_id: int, appliance id.
    Returns:
        pandas.DataFrame, ground truth appliance status.
    """
    query = """
    SELECT * FROM {}.ApplianceStatusGroundTruth
    WHERE appliance_id = {}
    ORDER BY time
    """.format(dataset, appliance_id)
    return pandas_gbq.read_gbq(query,
                               project_id,
                               index_col='time',
                               dialect='legacy')

def load_data(project_id, dataset, app_ids):
    """Load ground-truth data from big query table.

    Load true appliance status from big query data.

    Args:
        project_id: str, google project id.
        dataset: str, dataset name.
        app_ids: list, appliances' ids.
    Returns:
        dict, {appliance_id: ground truth data (pandas.DataFrame)}
    """
    print('Loading ground truth data ...')
    gt = {}
    for k in app_ids:
        gt[k] = load_ground_truth(project_id, dataset, k)
    print('Data loaded.')
    return gt

In [ ]:
# @title Load appliance info

app_info = get_appliance_info(GOOGLE_CLOUD_PROJECT, DATASET_ID)
app_info

,appliance_name,appliance_id
0,running_machine,0
1,washing_machine,1
2,dish_washer,2
3,microwave,3
4,toaster,4
5,kettle,5
6,rice_cooker,6
7,cooker,7


In [ ]:
# @title Create appliance Id to name mapping

app_id_name_map = {row[1]: row[0] for i, row in app_info.iterrows()}
app_id_name_map

{0: 'running_machine',
 1: 'washing_machine',
 2: 'dish_washer',
 3: 'microwave',
 4: 'toaster',
 5: 'kettle',
 6: 'rice_cooker',
 7: 'cooker'}

In [ ]:
# @title Load ground truth data (this can take some time)
ground_truth = load_data(project_id=GOOGLE_CLOUD_PROJECT,
                         dataset=DATASET_ID,
                         app_ids=app_id_name_map.keys())

Loading ground truth data ...
Data loaded.


# Start subscriber

When the following cell is run, it will pull messages from a pub/sub topic.  
The message contains both raw power readings and prediction results from CMLE.  
It plots the raw active powers, it also updates precision/recall for each appliance.  

In [ ]:
# @title Utility

class MsgProcesser(object):
    """Subscribe to a pub/sub topic and process incoming messages."""
    
    def __init__(self,
                 project_id,
                 ground_truth,
                 topic_name,
                 subscription_name,
                 app_id_name_map,
                 target_device):
        # member initialization
        self._gt = ground_truth
        self._data = pd.Series(np.zeros(self._gt[0].shape[0]),
                                index=self._gt[0].index)
        self._app_id_to_name = app_id_name_map
        self._target_device = target_device
        self._app_names = [app_id_name_map[k]
                           for k in range(len(app_id_name_map.keys()))]
        self._queue = queue.Queue(maxsize=100000)
        # create subsciprtion
        self._subscriber, self._subscription_path = (
            self.create_subscription(project_id, topic_name, subscription_name))
        #self._subscriber.subscribe(self._subscription_path,
            #                       callback=self._msg_callback)
        
    def create_subscription(self, project_id, topic_name, subscription_name):
        """Create a subscription in pub/sub.
    
        Before listening to incoming messages, we need to define a subscription.
        This function creates a new subscription only if it does not exist.

        Args:
          project_id: str, google cloud project id.
          topic_name: str, topic name.
          subscription_name: str, name of the new subscription.
        Returns:
          (google.cloud.pubsub_v1.SubscriberClient, topic_path)
        """
        print('Creating subscription "{}" to topic "{}" ...'.format(
            subscription_name, topic_name))
        subscriber = None
        subscription_path = None
        try:
            subscriber = pubsub_v1.SubscriberClient()
            topic_path = subscriber.topic_path(project_id, topic_name)
            subscription_path = subscriber.subscription_path(
                project_id, subscription_name)
            # if the subscription exists, exception is raised here
            subscription = subscriber.create_subscription(
                subscription_path, topic_path)
            print('Subscription created: {}'.format(subscription))
        except Exception as e:
            print('Subscription "{}" existed.'.format(subscription_name))
        return subscriber, subscription_path
    
    def async_pull_msg(self):
        """Pull messages asynchronously from pub/sub topic."""
        # initialize metrics
        self._metrics = [{'TP': 0, 'TN': 0, 'FP': 0, 'FN': 0}
                         for k in range(len(self._app_id_to_name))]
        precisions = [0. for k in self._app_id_to_name]
        recalls = [0. for k in self._app_id_to_name]
        # initialize UI
        fig, ax = plt.subplots(1, 1, figsize=(16, 4))
        sns.axes_style('white')
        # receive data and update UI
        print('Listening for messages on {} ...'.format(
          self._subscription_path))
        max_t = None
        while(True):
            # collect data from callback threads
            t, d, probs = self._queue.get()
            if max_t is None:
                max_t = t
            else:
                max_t = max(t, max_t)
            self._data.at[t] = d
            for k in self._app_id_to_name:
                truth = self._gt[k].status.at[t]
                pred = 1 if probs[k] >= 0.5 else 0
                if truth == 1 and pred == 1:
                    self._metrics[k]['TP'] += 1
                elif truth == 0 and pred == 0:
                    self._metrics[k]['TN'] += 1
                elif truth == 1 and pred == 0:
                    self._metrics[k]['FN'] += 1
                else:
                    self._metrics[k]['FP'] += 1
                TP = self._metrics[k]['TP']
                FP = self._metrics[k]['FP']
                FN = self._metrics[k]['FN']
                precisions[k] = (
                  np.nan if TP + FP == 0 else round(1.0 * TP / (TP + FP), 2))
                recalls[k] = (
                  np.nan if TP + FN == 0 else round(1.0 * TP / (TP + FN), 2))
                
            score_matrix = pd.DataFrame({'Precision': precisions,
                                         'Recall': recalls},
                                        index=self._app_names)
                        
            # update view in the main thread
            ax.clear()
            x = max_t.astype('datetime64[h]')
            mask = (self._data.index >= x) & (self._data.index < x + 1)
            ax = sns.lineplot(data=self._data[mask], linewidth=2.5, ax=ax)
            ax.set_xlabel('time')
            ax.set_ylabel('active power')
            ax.set_ylim(bottom=0)
            title = 'Device: {}, Date: {} (UTC)'.format(
              self._target_device, max_t.astype('datetime64[D]'))
            ax.set_title(title)
            display.clear_output(wait=True)
            display.display(plt.gcf())
            display.display(score_matrix)
            
    def _msg_callback(self, message):
        """Pub/sub pull callback."""
        try:
            data = json.loads(message.data.decode('utf-8'))
            device_id = data['device_id']
            if device_id == self._target_device:
              t = np.datetime64(data['time'][-1])
              d = data['data'][-1]
              probs = data['probs']
              self._queue.put((t, d, probs))
        except Exception as e:
            print('Error: {}'.format(e))
        message.ack()

In [ ]:
# @title Visualization

tt = MsgProcesser(project_id=GOOGLE_CLOUD_PROJECT,
                  ground_truth=ground_truth,
                  topic_name=PRED_TOPIC,
                  subscription_name=SUB_NAME,
                  app_id_name_map=app_id_name_map,
                  target_device=DEVICE_ID)
tt.async_pull_msg()

Creating subscription "sub1" to topic "pred" ...
Subscription "sub1" existed.


AttributeError: ignored